In [1]:
import numpy as np

from nnadvdet.core.maxprob.maxprob import MaxProb
from nnadvdet.core.ginicoef.ginicoef import DeepGini
from nnadvdet.core.reliable_bound import ReliableBound
from torchvision import transforms
from util.custom_transform import Translation2D
from copy import deepcopy
from torch.utils.data import DataLoader
from util.load_data import SelectDataset
import torch
from tqdm import tqdm

In [2]:
config = {
    'num_classes': 10,
    'ReliableBound': {
        'occ_name': 'ocsvm',
        'occ_config': {
            'epochs': 1000,
            'batch_size': 100000,
            'nu': 0.1,
            'scale': True
        }
    }
}

In [3]:
def build_translate_loader(input_size, factor, dataset='MNIST', batch_size=1000):
    translation_range = int(input_size * factor)
    loaders = {}
    for i in range(-translation_range, translation_range+1):
        for j in range(-translation_range, translation_range+1):
            trans_t = transforms.Compose([
                transforms.ToTensor(),
                transforms.Resize(input_size),
                Translation2D(i, j)
            ])
            data_t = SelectDataset(dataset, None, True, transform=trans_t)
            loaders[f'{i}{j}'] = DataLoader(data_t.dataset, batch_size=batch_size, shuffle=False, num_workers=4)
    return loaders

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model_name = 'LeNet5'
num_classes = 10
input_size = 32
model = torch.load('../checkpoints/mnist_lenet5.pth')
model.to(device)
model.eval()
loader_set = build_translate_loader(input_size, 0.1, batch_size=1000)

In [5]:
logits = []
preds = []
label_collect = []

In [6]:
pbar = tqdm(total=len(loader_set)*len(loader_set['00']))
for idx, loader in enumerate(loader_set):
    for inputs, labels in loader_set[loader]:
        inputs = inputs.to(device)
        labels = labels.to(device)

        logit_tmp = model(inputs)
        _, pred_tmp = torch.max(logit_tmp, 1)
        label_tmp = labels.data

        logits.extend(deepcopy(logit_tmp.detach().cpu().numpy().tolist()))
        preds.extend(deepcopy(pred_tmp.cpu().numpy().tolist()))
        label_collect.extend(deepcopy(label_tmp.cpu().numpy().tolist()))
        pbar.update(1)
pbar.close()

100%|██████████| 2940/2940 [03:34<00:00, 13.72it/s]


In [7]:
logits = np.asarray(logits, dtype=np.float32)
preds = np.asarray(preds, dtype=int)
label_collect = np.asarray(label_collect, dtype=int)

In [8]:
print(len([logits]), [logits][0].ndim, [logits][0].shape[-1])

1 2 10


In [9]:
adv_det0 = MaxProb(config)
adv_det1 = DeepGini(config)
adv_det2 = ReliableBound(config)

In [10]:
adv_det0.build_detector([logits], preds, label_collect)

2022-03-24 12:05:29.821 | INFO     | nnadvdet.core.maxprob.maxprob:build_detector:13 - MaxProb build finished.


In [11]:
adv_det1.build_detector([logits], preds, label_collect)

2022-03-24 12:05:29.824 | INFO     | nnadvdet.core.ginicoef.ginicoef:build_detector:13 - DeepGini build finished.


In [12]:
adv_det2.build_detector([logits], preds, label_collect)

2022-03-24 12:05:29.874 | INFO     | nnadvdet.core.reliable_bound.ocsvm:train:28 - Build start......
 83%|████████▎ | 2100000000/2530080000 [01:32<00:18, 22711131.34it/s]
2022-03-24 12:07:02.341 | INFO     | nnadvdet.core.reliable_bound.ocsvm:train:30 - Calculating boundaries for each class......
2022-03-24 12:07:04.081 | INFO     | nnadvdet.core.reliable_bound.ocsvm:train:32 - Build finished......
2022-03-24 12:07:04.083 | INFO     | nnadvdet.core.reliable_bound.reliable_bound:build_detector:35 - 


In [13]:
trans_t2 = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize(input_size)
])
data_t2 = SelectDataset('MNIST', None, False, transform=trans_t2)
loader_test = DataLoader(data_t2.dataset, batch_size=1000, shuffle=False, num_workers=4)
logits_test = []
preds_test = []
label_test = []
pbar2 = tqdm(total=len(loader_test))
for idx, loader in enumerate(loader_test):
    for inputs, labels in loader_test:
        inputs = inputs.to(device)
        labels = labels.to(device)

        logit_tmp = model(inputs)
        _, pred_tmp = torch.max(logit_tmp, 1)
        label_tmp = labels.data

        logits_test.extend(deepcopy(logit_tmp.detach().cpu().numpy().tolist()))
        preds_test.extend(deepcopy(pred_tmp.cpu().numpy().tolist()))
        label_test.extend(deepcopy(label_tmp.cpu().numpy().tolist()))
        pbar2.update(1)
pbar2.close()
logits_test = np.asarray(logits_test, dtype=np.float32)
preds_test = np.asarray(preds_test, dtype=int)
label_test = np.asarray(label_test, dtype=int)
flag_test = np.zeros(logits_test.shape[0], dtype=int)
flag_test[preds_test==label_test] = 1

100it [00:06, 15.69it/s]                      


In [14]:
result0 = adv_det0.batched_detect([logits_test])
result1 = adv_det1.batched_detect([logits_test])

In [15]:
print(result0.shape, result1.shape)

(100000,) (100000,)


In [16]:
from sklearn.metrics import roc_curve, auc
fpr0, tpr0, thres0 = roc_curve(flag_test, result0)
fpr1, tpr1, thres1 = roc_curve(flag_test, result1)
print(auc(fpr0, tpr0))
print(auc(fpr1, tpr1))

0.9789382151525658
0.9790754666551086


In [17]:
result2 = []
for i in range(logits_test.shape[0]):
    result2.append(adv_det2.detect(logits_test[i].reshape(1, -1), x_pred=preds_test[i]))
result2 = np.asarray(result2)

In [18]:
print(result2[flag_test==1].mean(), result2[flag_test==0].mean())

0.7621816098580851 0.6324449045488738


In [19]:
fpr2, tpr2, thres2 = roc_curve(flag_test, result2)
print(auc(fpr2, tpr2))

0.8369966842926491
